In [1]:
SCHOOL="dartmouth"
SUBJECT="opinion"
START_YEAR=2010
FINAL_YEAR=2022
TOPIC_EMBEDDING_MODEL = "all-MiniLM-L6-v2" # alt: "all-mpnet-base-v2"
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [13]:
import sys
sys.path.append('src')
import ouraws
import nltk
import ourgraphs
import textutil

In [5]:
%pwd

'/Users/anishka/Documents/GitHub/diversity-colleges/src'

In [8]:
df = pd.read_parquet("data/pomona-opinions-SNAPSHOT.parquet")

In [9]:
df.shape

(1718, 6)

In [10]:
df["year"].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010])

In [14]:
RESULTS_OUTPUT = f"{OUTPUT_DIR}/{SCHOOL}-{SUBJECT}-diversity-results.parquet"

In [15]:
import requests
import os

BIPARTISAN_API_KEY = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [16]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-1.489565


In [17]:

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")

Year: 2010 ... 131
2010	131	-110.812606	-0.8458977557251909
Year: 2011 ... 125
2011	125	-392.36253500000004	-3.13890028
Year: 2012 ... 133
2012	133	-424.74822999999986	-3.1935957142857134
Year: 2013 ... 165
2013	165	-588.8014300000001	-3.568493515151516
Year: 2014 ... 150
2014	150	-406.43686799999995	-2.7095791199999995
Year: 2015 ... 125
2015	125	-550.1876749999999	-4.401501399999999
Year: 2016 ... 125
2016	125	-740.9412189999995	-5.927529751999995
Year: 2017 ... 113
2017	113	-964.5114180000003	-8.535499274336285
Year: 2018 ... 120
2018	120	-1018.052546	-8.483771216666666
Year: 2019 ... 151
2019	151	-888.0072990000006	-5.880843039735103
Year: 2020 ... 127
2020	127	-899.9752460000003	-7.086419259842522
Year: 2021 ... 132
2021	132	-893.8355670000001	-6.771481568181819
Year: 2022 ... 116
2022	116	-650.7396160000001	-5.60982427586207


In [18]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)